In [ ]:
import os
import dotenv

dotenv.load_dotenv()
MAIN_SAFE = "0x4c190e3C582e877942007Db3009feACc1Da6C481"
METAMASK_EOA = os.getenv("WALLET_ADDRESS")
POLYMARKET_SAFE = "0x2617746e3Da3cC26EeD139d873f0A05137023e40"
TRADER_EOA_PRIVATE_KEY = os.getenv("TRADER_EOA_PRIVATE_KEY")

ACC_2_ADDRESS = "0xfd4fac895d20912b5d3abdd8695d891baa5e90d5"
ACC_2_PRIVATE_KEY = os.getenv("ACC_2_PRIVATE_KEY")
USDCE_ADDRESS = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"
PRIVATE_KEY = os.getenv("PRIVATE_KEY")  

POLYGON_RPC = os.getenv("POLYGON_RPC")  # Polygon rpc url

In [ ]:
import json
import time
from web3 import Web3
from eth_account import Account
from eth_account.messages import encode_defunct

# ---------- Configuration (CHANGE THESE) ----------
RPC_URL = POLYGON_RPC  # Your Polygon RPC (or 1rpc.io/matic, Infura, Alchemy, etc.)
SAFE_ADDRESS = MAIN_SAFE
CHAIN_ID = 137  # Polygon mainnet chain id
# Target contract (the one whose redeemPositions we call)
TARGET_CONTRACT = "0x4D97DCd97eC945f40cF65F87097ACe5EA0476045"
# Parameters for redeemPositions — replace with real values:
COLLATERAL_TOKEN = (
    "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"  # Example: USDC.e on Polygon
)
PARENT_COLLECTION_ID = (
    "0x0000000000000000000000000000000000000000000000000000000000000000"
)
CONDITION_ID = "0xb72502347b6186d305e49add224b07e4ed01399e09320f95070e6d16eb773d5d"
INDEX_SETS = [1]  # Example index sets (uint256[])

# Gas / Safe params (tune as needed)
SAFE_TX_GAS = 0  # let the Safe estimate (0 often acceptable)
BASE_GAS = 0
GAS_PRICE = 0
GAS_TOKEN = "0x0000000000000000000000000000000000000000"  # native token (MATIC)
REFUND_RECEIVER = "0x0000000000000000000000000000000000000000"

# ---------------------------------------------------
# --- MUST be 32-byte hex string ---
PRIVATE_KEY = PRIVATE_KEY
acct = Account.from_key(PRIVATE_KEY)
owner = acct.address
w3 = Web3(Web3.HTTPProvider(RPC_URL))

print("Owner:", owner)

# --- ABI fragments ---
safe_abi = [
    {
        "type": "function",
        "name": "getTransactionHash",
        "stateMutability": "view",
        "inputs": [
            {"name": "to", "type": "address"},
            {"name": "value", "type": "uint256"},
            {"name": "data", "type": "bytes"},
            {"name": "operation", "type": "uint8"},
            {"name": "safeTxGas", "type": "uint256"},
            {"name": "baseGas", "type": "uint256"},
            {"name": "gasPrice", "type": "uint256"},
            {"name": "gasToken", "type": "address"},
            {"name": "refundReceiver", "type": "address"},
            {"name": "nonce", "type": "uint256"},
        ],
        "outputs": [{"name": "txHash", "type": "bytes32"}],
    },
    {
        "type": "function",
        "name": "execTransaction",
        "stateMutability": "nonpayable",
        "inputs": [
            {"name": "to", "type": "address"},
            {"name": "value", "type": "uint256"},
            {"name": "data", "type": "bytes"},
            {"name": "operation", "type": "uint8"},
            {"name": "safeTxGas", "type": "uint256"},
            {"name": "baseGas", "type": "uint256"},
            {"name": "gasPrice", "type": "uint256"},
            {"name": "gasToken", "type": "address"},
            {"name": "refundReceiver", "type": "address"},
            {"name": "signatures", "type": "bytes"},
        ],
        "outputs": [{"name": "success", "type": "bool"}],
    },
    {
        "type": "function",
        "name": "nonce",
        "stateMutability": "view",
        "inputs": [],
        "outputs": [{"name": "nonce", "type": "uint256"}],
    },
]

redeem_abi = [
    {
        "type": "function",
        "name": "redeemPositions",
        "stateMutability": "nonpayable",
        "inputs": [
            {"name": "collateralToken", "type": "address"},
            {"name": "parentCollectionId", "type": "bytes32"},
            {"name": "conditionId", "type": "bytes32"},
            {"name": "indexSets", "type": "uint256[]"},
        ],
        "outputs": [],
    }
]


target = w3.eth.contract(TARGET_CONTRACT, abi=redeem_abi)
safe = w3.eth.contract(SAFE_ADDRESS, abi=safe_abi)

# ----- Encode redeemPositions -----
conditionId = Web3.to_bytes(hexstr=CONDITION_ID)
len(conditionId)  # → 32

calldata = target.encode_abi(
    "redeemPositions",
    args=[
        "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174",
        bytes.fromhex("00" * 32),
        conditionId,
        [1],
    ],
)


data_bytes = bytes.fromhex(calldata[2:])

nonce = safe.functions.nonce().call()

safe_tx_hash = safe.functions.getTransactionHash(
    TARGET_CONTRACT,
    0,
    data_bytes,
    0,  # operation = CALL
    0,
    0,
    0,
    "0x0000000000000000000000000000000000000000",
    "0x0000000000000000000000000000000000000000",
    nonce,
).call()

print("Safe Tx Hash:", safe_tx_hash.hex())

# ----- SIGN RAW 32-byte HASH -----
# signed = acct.sign_hash(safe_tx_hash)
# signature = signed.signature
signed = Account._sign_hash(safe_tx_hash, private_key=PRIVATE_KEY)

signature = signed.signature  # r || s || v (65 bytes)

print("Signature:", signature.hex())

# ----- Execute via Safe -----
tx = safe.functions.execTransaction(
    TARGET_CONTRACT,
    0,
    data_bytes,
    0,
    0,
    0,
    0,
    "0x0000000000000000000000000000000000000000",
    "0x0000000000000000000000000000000000000000",
    signature,
)

tx_data = tx.build_transaction(
    {
        "from": owner,
        "nonce": w3.eth.get_transaction_count(owner),
        "gasPrice": w3.eth.gas_price,
    }
)

tx_data["gas"] = int(w3.eth.estimate_gas(tx_data) * 1.2)

signed_send = acct.sign_transaction(tx_data)
tx_hash = w3.eth.send_raw_transaction(signed_send.raw_transaction)

print("Sent:", tx_hash.hex())